Work on getting an IGC that models ecostress
============================================

In [1]:
from geocal import *
from ecostress import *
import h5py
import os
import scipy
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
orb_fname = "/data/smyth/ecostress-test-data/latest/L1A_RAW_ATT_80005_20150124T204251_0100_01.h5.expected"
orb = HdfOrbit_Eci_TimeJ2000(orb_fname, "", "Ephemeris/time_j2000",
                             "Ephemeris/eci_position",
                             "Ephemeris/eci_velocity",
                             "Attitude/time_j2000",
                             "Attitude/quaternion")
rad_fname = "/data/smyth/ecostress-test-data/latest/ECOSTRESS_L1B_RAD_80005_001_20150124T204251_0100_01.h5.expected"
f = h5py.File(rad_fname, "r")
tmlist = f["/Time/line_start_time_j2000"][:]
tt = MeasuredTimeTable([Time.time_j2000(t) for t in tmlist])
datum = os.environ["AFIDS_VDEV_DATA"] + "/EGM96_20_x100.HLF"
srtm_dir = os.environ["ELEV_ROOT"]
dem = SrtmDem(srtm_dir,False, DatumGeoid96(datum))

In [3]:
# Kind of round about way to come up with a time table, we'll likely want
# to change this in the future
f = h5py.File(rad_fname, "r")
tmlist = f["/Time/line_start_time_j2000"][:]
tt = MeasuredTimeTable([Time.time_j2000(t) for t in tmlist])
t0, fc = tt.time(ImageCoordinate(0,0))
tt = EcostressTimeTable(t0, False)

In [4]:
cam = read_shelve("camera.xml")
sm = EcostressScanMirror()

In [5]:
igc = EcostressImageGroundConnection(orb, tt, cam, sm, dem, None)
write_shelve("igc.xml", igc)

In [50]:
igc = read_shelve("igc.xml")

In [51]:
g1 = igc.ground_coordinate(ImageCoordinate(0,0))
g2 = igc.ground_coordinate(ImageCoordinate(0,1))
g3 = igc.ground_coordinate(ImageCoordinate(1,0))
print(distance(g1,g2))
print(distance(g1,g3))

85.18372616177685
43.092950045954694


In [18]:
def compare_ref(ic,b):
    igc.band = 4
    gc = igc.ground_coordinate(ic)
    igc.band = b
    ic, status = igc.image_coordinate_with_status(gc)
    igc.band = 4
    if(status):
        return ic
    else: 
        return None

In [21]:
pt_list = []
for i in range(0,igc.number_line,100):
    for j in range(0,igc.number_sample,100):
        ic = compare_ref(ImageCoordinate(i,j), 2)
        if(ic):
            pt_list.append((ImageCoordinate(i,j), ic))

In [25]:
gtp = GeometricTiePoints()
for ic_resampled, ic_original in pt_list:
    gtp.add_point(ic_resampled, ic_original)

In [26]:
gm = QuadraticGeometricModel()
gm.fit_transformation(gtp)

In [29]:
print(gm.resampled_image_coordinate(pt_list[0][1]))

(530.347, 99.4716)


In [30]:
print(pt_list[0][0])

(0, 100)


In [31]:
gtp = GeometricTiePoints()
for i in range(0,256,20):
    for j in range(0,igc.number_sample,100):
        ic = compare_ref(ImageCoordinate(i,j), 2)
        if(ic):
            gtp.add_point(ImageCoordinate(i,j),ic)

In [32]:
gm = QuadraticGeometricModel()
gm.fit_transformation(gtp)

In [41]:
for i in range(1000):
    print(igc.image_coordinate(igc.ground_coordinate(ImageCoordinate(i,100))))

(0.0013507, 100.003)
(1.00145, 100.003)
(2.00155, 100.003)
(3.00164, 100.003)
(4.00159, 100.006)
(5.00183, 100.003)
(6.00178, 100.005)
(7.00187, 100.005)
(8.00211, 100.002)
(9.00205, 100.005)
(10.0021, 100.004)
(11.0024, 100.002)
(12.0025, 100.001)
(13.0025, 100.001)
(14.0026, 100.001)
(15.0026, 100.003)
(16.0026, 100.003)
(17.0028, 100)
(18.0029, 100)
(19.003, 99.9999)
(20.0028, 100.005)
(21.0029, 100.005)
(22.0029, 100.005)
(23.003, 100.004)
(24.0032, 100.002)
(25.0033, 100.001)
(26.0032, 100.004)
(27.0034, 100.001)
(28.0033, 100.003)
(29.0035, 100)
(30.0034, 100.003)
(31.0035, 100.003)
(32.0036, 99.9997)
(33.0037, 99.9994)
(34.0037, 99.9992)
(35.0038, 99.999)
(36.0037, 100.001)
(37.0037, 100.001)
(38.0038, 100.001)
(39.0039, 99.998)
(40.0038, 100.001)
(41.0039, 100)
(42.004, 99.9973)
(43.004, 99.9971)
(44.0041, 99.9969)
(45.0041, 99.9966)
(46.0041, 99.9964)
(47.0041, 99.9961)
(48.0039, 100.001)
(49.004, 99.9984)
(50.004, 100.001)
(51.004, 100.001)
(52.004, 100)
(53.0041, 99.9975)
(5

In [52]:
print(Geodetic(igc.ground_coordinate(ImageCoordinate(0,0))))
print(Geodetic(igc.ground_coordinate(ImageCoordinate(255,0))))
print(Geodetic(igc.ground_coordinate(ImageCoordinate(256,0))))

Geodetic: (37.7424 deg, -124.63 deg, -37.4919 m)
Geodetic: (37.6812 deg, -124.532 deg, -37.5252 m)
Geodetic: (37.6973 deg, -124.558 deg, -37.5163 m)


In [54]:
print(igc.image_coordinate(igc.ground_coordinate(ImageCoordinate(255,0))))

(323.203, 0.620259)


In [48]:
print(Geodetic(igc.ground_coordinate(ImageCoordinate(240,0))))

Geodetic: (37.7267 deg, -124.636 deg, -37.5145 m)


In [49]:
print(distance(igc.ground_coordinate(ImageCoordinate(240,0)),
               igc.ground_coordinate(ImageCoordinate(256,0))))

18556.370148664973
